In [ ]:
import numpy as np
from multi_step import *
from utils import objective, stage

Using backend: tensorflow


Case study based on the van de Vusse reactions in a semi-batch process

A -> B -> C
2A -> D

https://pubs.acs.org/doi/pdf/10.1021/i160048a008

In [ ]:
# Generate conditions to be used for generating data

# Define the condition ranges [lower,upper,step (if applicable)]
#[P1, P2, P3, V0, sigmaF]
condition_ranges = [[0, 10],[0, 10],[0,10],[0,1],[0,0.5]]

# Generate all condition permutations
# conditions = toy.allConditions(condition_ranges)

# Generate values to explore the search space using low discrepancy
# quasi-random numbers and discrete methods
num_points = 1000
conditions =  toy.sobolSequenceConditions(condition_ranges, num_points)

In [ ]:
# Generate the yields of the various materials
yieldList = []
costList = []
for conditions_run in conditions:
    yields, costs = toy.experimentalRun(1,conditions_run)
    yieldList.append(yields)
    costList.append(costs)
yieldList = np.array(yieldList)

In [ ]:
# Fit the first stage data
x1 = conditions[:,:3]
y1 = yieldList[:,2:5]
toy.fit_stage_1([x1,y1], epochs=5)

In [ ]:
# Fit the second stage data

x2 = np.concatenate((np.expand_dims(yieldList[:,3],axis=1),conditions[:,3:6]),axis=1)
y2 = yieldList[:,5:9]
toy.fit_stage_2([x2,y2], epochs=5)

In [ ]:
# Fit the third stage data
x3 = np.concatenate((np.expand_dims(yieldList[:,4],axis=1),yieldList[:,7:9],conditions[:,6:]),axis=1)
y3 = yieldList[:,9:]
toy.fit_stage_3([x3,y3], epochs=5)

In [ ]:
#Plot stage 1
# num_plot_points = 20
# plot_var = 0
# plot_x = np.hstack((np.expand_dims(np.linspace(2,10,num_plot_points),1),np.full((num_plot_points,1),7),np.full((num_plot_points,1),5)))
# plot_y = np.array([toy.stage_1(0.4, *row) for row in plot_x])
# toy.stage_1_model.visualise(plot_x,plot_y,plot_x,plot_var,'Stage_1')

In [ ]:
# # Save the models
# toy.stage_1_model.save('../examples/Toy_problem/Saved_Models/stage_1_model')
# toy.stage_2_model.save('../examples/Toy_problem/Saved_Models/stage_2_model')
# toy.stage_3_model.save('../examples/Toy_problem/Saved_Models/stage_3_model')

In [ ]:
# Generate the multi-step class
multi_step = multi_step('adam')

In [ ]:
# Load the models from a save
# import tensorflow as tf
# toy = ts.Toy()
# 
# toy.stage_1_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_1_model.h5')
# toy.stage_2_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_2_model.h5')
# toy.stage_3_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_3_model.h5')

In [ ]:
# Load the models into the framework

#Define the stages
# stageID, stageModel, conditionRanges, followingStages (followingStageID, feedingVariables)
stage1 = stage(0, toy.stage_1_model, condition_ranges[0:3], [[1, [1]], [2, [2]]])
stage2 = stage(1, toy.stage_2_model, condition_ranges[3:6], [[2,[2,3]]])
stage3 = stage(2, toy.stage_3_model, condition_ranges[6:])

stages = [stage1, stage2, stage3]

multi_step.loadModels(stages)

In [ ]:
# Define the objective functions
def objectiveFunction1(var1, var2, var3):
    return (var1+var2)**2+(var2+var3)**2
    
def objectiveFunction2(var1, var2, var3, var4):
    return (var1+var4)**2-var3**2+var2**2

# Define the objective objects including the objective functions and the IDs of relevant variables/outputs
# Define objective variables with form: [stage, 'input'/'output', stage variable]
objective1 = objective(objectiveFunction1, [[stage1,'inputs',1],[stage2,'inputs',1],[stage3,'inputs',1]])
objective2 = objective(objectiveFunction2,[[stage1,'outputs',1],[stage2,'outputs',0],[stage3,'inputs',1],[stage2,'outputs',1]])

objectives = [objective1, objective2]

In [ ]:
# Define the objectives for the framework
multi_step.defineObjectives(objectives)

In [ ]:
# Optimise the process
# numObjectives, numObjectiveSamples, preferenceCodeDim, hyperparameterGeneratorDim, epochs=5, h_dim=5, n_layers=4, network_type='GATConv', **kwargs
multi_step.train(2, 4, 10, 100, epochs=100, h_dim=5, n_layers=4, network_type='GATConv', num_heads=3)